In [1]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
from teamName import getMyPosition as getPosition

nInst = 0
nt = 0
commRate = 0.0010
dlrPosLimit = 10000

def loadPrices(fn):
    global nt, nInst
    #df=pd.read_csv(fn, sep='\s+', names=cols, header=None, index_col=0)
    df=pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    nt, nInst = df.values.shape
    return (df.values).T

pricesFile="./data/prices.txt"
prcAll = loadPrices(pricesFile)
print ("Loaded %d instruments for %d days" % (nInst, nt))

currentPos = np.zeros(nInst)

pred_df = pd.read_csv('pred.csv')


def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_,nt) = prcHist.shape
    for t in range(1+250,251+250): 
        prcHistSoFar = prcHist[:,:t]
        newPosOrig = getPosition(pred_df, t-1-250)
        curPrices = prcHistSoFar[:,-1] #prcHist[:,t-1]
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        clipPos = np.clip(newPosOrig, -posLimits, posLimits)
        newPos = np.array([np.trunc(x) for x in clipPos])
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
        print ("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" % (t,value, todayPL, totDVolume, ret))
    pll = np.array(todayPLL)
    (plmu,plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(250) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)



(meanpl, ret, plstd, sharpe, dvol) = calcPL(prcAll)
score = meanpl - 0.1*plstd
print ("=====")
print ("mean(PL): %.1lf" % meanpl)
print ("return: %.5lf" % ret)
print ("StdDev(PL): %.2lf" % plstd)
print ("annSharpe(PL): %.2lf " % sharpe)
print ("totDvolume: %.0lf " % dvol)
print ("Score: %.2lf" % score)




Loaded 50 instruments for 500 days
Day 251 value: -9.98 todayPL: $-9.98 $-traded: 9984 return: -0.00100
Day 252 value: -5.98 todayPL: $4.00 $-traded: 9984 return: -0.00060
Day 253 value: -1.98 todayPL: $4.00 $-traded: 9984 return: -0.00020
Day 254 value: -17.95 todayPL: $-15.97 $-traded: 29953 return: -0.00060
Day 255 value: -41.95 todayPL: $-24.00 $-traded: 49939 return: -0.00084
Day 256 value: -57.92 todayPL: $-15.97 $-traded: 69917 return: -0.00083
Day 257 value: -81.90 todayPL: $-23.98 $-traded: 89886 return: -0.00091
Day 258 value: -81.90 todayPL: $0.00 $-traded: 89886 return: -0.00091
Day 259 value: -97.85 todayPL: $-15.95 $-traded: 109848 return: -0.00089
Day 260 value: -97.85 todayPL: $0.00 $-traded: 109848 return: -0.00089
Day 261 value: -97.85 todayPL: $0.00 $-traded: 109848 return: -0.00089
Day 262 value: -101.86 todayPL: $-4.01 $-traded: 109848 return: -0.00093
Day 263 value: -125.84 todayPL: $-23.98 $-traded: 129819 return: -0.00097
Day 264 value: -141.81 todayPL: $-15.97 